Zunächst müssen wir das OdeNet-Modul importieren

In [1]:
from odenet import *


Mit OdeNet.word_info() greift man auf Lexikon-Einträge zu, bekommt die Lexikon-ID für ein Wort, den Lemma-Wert, POS und die IDs der Synsets, in denen das Wort enthalten ist.

In [9]:
OdeNet.word_info("wenigst")

wenigst a 
------------------------
SENSE: odenet-1914-a
ILI: i8207
DEFINITION: the least possible
DE_DEFINITION: []
WORDS: ['kleinst', 'geringst', 'wenigst', 'minimalst']
RELATIONS: 
------------------------
LEMMA: wenigst
POS: a
SENSE ID: w9329_1914-a


Hier bekommt man die Lexikon-IDs für eine Liste von Wörtern.

In [3]:
wordlist = ['Frühling','Sommer','Herbst','Winter']
for w in wordlist:
    print(w + ': ' + OdeNet.word_id(w))

Frühling: w14145
Sommer: w44811
Herbst: w1202374
Winter: w25612


Mit check_word_id bekommt man für eine Lexikon-ID Lemma, POS, Synsets und Relationen

In [4]:
check_word_id('w14145')

LEMMA: Frühling
POS: n
SENSE ID: w14145_3067-n


Mit words_in_synset bekommt man die Wörter, die in einem Synset sind.

In [2]:
words_in_synset('odenet-362363-n')

['Web-Suchmaschine', 'Internet-Suchmaschine']

Mit check_synset bekommt man alle Informationen zu einem Synset.

In [2]:
print(check_synset("odenet-10112-n"))
print(check_synset("odenet-2994-n"))
#print(check_synset("odenet-10706-n"))
#print(check_synset("odenet-12925-n"))
#print(check_synset("odenet-29326-n"))
#print(check_synset("odenet-3723-n"))
#print(check_synset("odenet-6183-n"))
#print(check_synset("odenet-28448-n"))
#print(check_synset("odenet-29006-n"))


('i69094', 'a rationalized mental attitude', [], [('hyponym', 'odenet-2994-n'), ('hyponym', 'odenet-5478-n')], ['Fassung', 'Haltung', 'Kontrolle', 'Beherrschung', 'Contenance'], ['odenet-10112-n', 'odenet-11492-n', 'odenet-12665-n', 'odenet-19674-n', 'odenet-22960-n', 'odenet-28117-n', 'odenet-3853-n', 'odenet-905-n'])
('i69168', 'a mental position from which things are viewed', [], [('hypernym', 'odenet-905-n'), ('hyponym', 'odenet-4991-n')], ['Option', 'Perspektive', 'günstige Gelegenheit', 'Weg', 'Gunst der Stunde', 'Chance', 'Möglichkeit', 'Aussicht'], ['odenet-24733-n', 'odenet-2994-n'])


In [6]:
hypernyms_word("Steuer")

[('w3305_664-n',
  'odenet-4311-n',
  ['Steuerung', 'Angleichung', 'Aussteuerung']),
 ('w3305_10289-n', 'odenet-31291-n', ['Heermannie', 'Heerbann']),
 ('w3305_24723-n', 'odenet-31291-n', ['Heermannie', 'Heerbann']),
 ('w3305_24723-n', 'odenet-9595-n', ['Preis', 'Gebühr', 'Abgabe', 'Taxe'])]

In [2]:
hypernyms_path("odenet-4634-n",[])

['odenet-10049-n', 'odenet-26421-n']

In [8]:
hyponyms_word("Veranstaltung")

[('w64697_17741-n',
  'odenet-14850-n',
  ['Lager',
   'Kinderlandverschickung',
   'Freizeit',
   'Ferienlager',
   'Ferienfreizeit']),
 ('w64697_17741-n',
  'odenet-17740-n',
  ['Auftritt',
   'Gig',
   'Live Event',
   'Live-Act',
   'Live-Veranstaltung',
   'Liveact',
   'Live-Gig',
   'Show',
   'Live-Auftritt',
   'Live-Entertainment']),
 ('w64697_17741-n',
  'odenet-17742-n',
  ['Tiershow', 'Tiertheater', 'Tierveranstaltung']),
 ('w64697_17741-n',
  'odenet-214-n',
  ['Besprechung', 'Consulting', 'Beratung', 'Konsultation', 'Gespräch']),
 ('w64697_17741-n', 'odenet-25683-n', ['Matinee', 'Vormittagsveranstaltung']),
 ('w64697_17741-n',
  'odenet-2801-n',
  ['Gespräch',
   'Austausch',
   'Unterhaltung',
   'Gedankenaustausch',
   'Zwiegespräch',
   'Unterredung',
   'Diskussion',
   'Zwiesprache',
   'Konversation',
   'Wortwechsel',
   'Aussprache',
   'Dialog']),
 ('w64697_17741-n',
  'odenet-32187-n',
  ['Vorkurs',
   'Einführungsveranstaltung',
   'Vorbereitungskurs',
   'Vor

In [9]:
meronyms_word("Morgenland")

[('w12715_2720-n',
  'odenet-14514-n',
  ['Saudi-Arabien', 'Königreich Saudi-Arabien', 'Saudisch-Arabien']),
 ('w12715_2720-n', 'odenet-14519-n', ['Syrien', 'Arabische Republik Syrien']),
 ('w12715_2720-n', 'odenet-14516-n', ['Türkei', 'Republik Türkei']),
 ('w12715_9897-n', 'odenet-9309-n', ['Ferner Osten', 'Fernost'])]

In [2]:
synonyms_word("Vogel")

[['Piepmatz', 'Federvieh', 'gefiederter Freund'],
 ['Flugzeug', 'Flieger', 'Kiste', 'Maschine', 'Aeroplan'],
 ['Type',
  'Sonderling',
  'Kauz',
  'seltsamer Heiliger',
  'Unikum',
  'komischer Kauz',
  'Wunderling',
  'seltsamer Kauz',
  'schrille Schraube',
  'seltsamer Patron',
  'skurriler Typ',
  'verrückte Person',
  'schräger Vogel',
  'bunter Vogel',
  '(ausgefallener, besonderer) Mensch',
  'Urvieh',
  'komischer Vogel',
  'schräger Fürst',
  'sonderbarer Zeitgenosse',
  'irrer Typ',
  'Original',
  'schrille Type',
  'verrücktes Huhn',
  'schrille Person',
  'Mensch, wie man ihn nicht alle Tage trifft',
  'bizarre Person',
  'merkwürdiger Zeitgenosse',
  'Freak',
  'ungewöhnlicher Mensch',
  'seltsame Type',
  'uriger Typ',
  'komischer Heiliger']]

In [10]:
myword = "zeigen"
(lemma_id, lemma_value, pos, senses) = check_word_lemma(myword)
print (lemma_value + " " + pos + " ")
for sense in senses:
    print("SENSE: " + str(sense[1]) + "  " + str(check_synset(sense[1])) + "\n")
print("HYPERNYMS: " + str(hypernyms_word(myword)))
print("HYPONYMS: " + str(hyponyms_word(myword)))
print("MERONYMS: " + str(meronyms_word(myword)))
print("HOLONYMS: " + str(holonyms_word(myword)))
print("ANTONYMS: " + str(antonyms_word(myword)))


zeigen v 
SENSE: odenet-533-v  ('i33216', [], 'Vorschlagen, etwas für jemanden zu tun.', [('hypernym', 'odenet-8644-v')], ['andienen', 'anbieten', 'bieten', 'zeigen'])

SENSE: odenet-3065-v  ('i32498', [], 'Jemanden etwas sehen lassen.', [('hypernym', 'odenet-9246-v')], ['zeigen', 'präsentieren', 'darstellen', 'vorzeigen', 'abbilden', 'vorführen', 'demonstrieren', 'vorstellen'])

SENSE: odenet-4206-v  ('i32440', [], 'Etwas öffentlich sagen.', [('hyponym', 'odenet-2339-v'), ('hyponym', 'odenet-11159-v'), ('hyponym', 'odenet-1192-v'), ('hyponym', 'odenet-22914-v'), ('hyponym', 'odenet-2770-v')], ['kundtun', 'zeigen', 'offenbaren', 'bekunden'])

SENSE: odenet-5328-v  ('i25048', [], 'Die Wahrheit von etwas deutlich machen.', [('hypernym', 'odenet-12792-v')], ['bestätigen', 'zeigen', 'aufzeigen', 'nachweisen', 'belegen', 'Nachweis erbringen', 'Beweis erbringen für', 'beweisen'])

SENSE: odenet-9990-v  ('i25049', [], 'Sich als etwas erweisen, offenbaren.', [], ['zeigen', 'niederschlagen', 'm